In [16]:
import pandas as pd
import numpy as np
from data_gen import DataGenerator

Нормальные X и y, слабые корреляции

In [17]:
gen1 = DataGenerator(
    n_samples=2000,
    distributions={
        "X1": {"type": "normal", "mean": 0, "std": 1},
        "X2": {"type": "bernoulli", "p": 0.5},
        "y0": {"type": "normal", "mean": 5, "std": 1},
    },
    time_correlations={"X1": 0.2, "X2": 0.1, "y0": 0.3},
    effect_size=2.0,
    seed=42
)
df1 = gen1.generate()

Высокая автокорреляция и гамма-распределение для Y

In [18]:
gen2 = DataGenerator(
    n_samples=2000,
    distributions={
        "X1": {"type": "normal", "mean": 1, "std": 2},
        "X2": {"type": "bernoulli", "p": 0.3},
        "y0": {"type": "gamma", "shape": 2.0, "scale": 3.0},
    },
    time_correlations={"X1": 0.8, "X2": 0.4, "y0": 0.9},
    effect_size=3.0,
    seed=123
)
df2 = gen2.generate()

Сильно смещенное распределение X1 и слабый эффект

In [19]:
gen3 = DataGenerator(
    n_samples=3000,
    distributions={
        "X1": {"type": "normal", "mean": 10, "std": 5},
        "X2": {"type": "bernoulli", "p": 0.7},
        "y0": {"type": "normal", "mean": 0, "std": 1},
    },
    time_correlations={"X1": 0.5, "X2": 0.4, "y0": 0.6},
    effect_size=0.5,
    seed=7
)
df3 = gen3.generate()

Сильный эффект и Bernoulli-переменные

In [20]:
gen4 = DataGenerator(
    n_samples=1500,
    distributions={
        "X1": {"type": "bernoulli", "p": 0.2},
        "X2": {"type": "bernoulli", "p": 0.6},
        "y0": {"type": "normal", "mean": 3, "std": 2},
    },
    time_correlations={"X1": 0.2, "X2": 0.5, "y0": 0.4},
    effect_size=7.0,
    seed=2024
)
df4 = gen4.generate()

Гамма Y, нормальный X1, умеренный эффект

In [21]:
gen5 = DataGenerator(
    n_samples=2500,
    distributions={
        "X1": {"type": "normal", "mean": 0, "std": 3},
        "X2": {"type": "bernoulli", "p": 0.5},
        "y0": {"type": "gamma", "shape": 5.0, "scale": 1.0},
    },
    time_correlations={"X1": 0.3, "X2": 0.2, "y0": 0.85},
    effect_size=4.0,
    seed=99
)
df5 = gen5.generate()

In [22]:
dfs = [df1, df2, df3, df4, df5]

In [29]:
df1

,X1,X1_lag,X2,X2_lag,y0,y0_lag_1,y0_lag_2,z,U,D,d,y1,y
0,-0.297307,-0.472199,0,1,4.794322,6.330246,5.055205,1,-0.904401,0,0,6.039658,4.794322
1,-1.464946,0.461551,0,0,4.088637,4.389171,6.652239,0,-0.815788,0,0,3.272692,4.088637
2,0.329456,0.033293,1,1,4.122169,2.984033,4.649830,0,0.649605,0,0,5.338270,4.122169
3,-1.708621,-0.737885,1,0,4.877743,7.188729,4.008481,1,-1.707343,0,0,4.046640,4.877743
4,0.020508,0.706798,1,0,2.998831,4.361080,5.076842,1,-0.584229,1,1,5.725767,5.725767
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1.606489,1.010382,0,1,7.125017,4.988984,5.526429,0,0.390870,1,0,6.338262,7.125017
1996,1.055169,2.005243,0,0,4.389471,4.024334,4.383882,1,-0.839797,0,0,2.960262,4.389471
1997,1.581608,1.617605,0,0,4.522303,5.596374,5.291108,1,-0.511105,0,0,7.301909,4.522303
1998,1.812040,1.406892,0,1,4.380839,4.439230,4.039406,0,-3.635200,0,0,1.954124,4.380839


Этап 1: сравнение нашей реализации CUPAC и сторонних решений.

In [23]:
from autocupac import CUPACTransformer

In [24]:
for df in dfs:
    transformer = CUPACTransformer(target_col='y')
    transformer.fit(df)
    transformed_data = transformer.transform(df)

    print(f'Сижение дисперсии: {transformer.variance_reduction}')

Сижение дисперсии: 9.498780357349295
Сижение дисперсии: 0.45465370191287935
Сижение дисперсии: 36.345233730811444
Сижение дисперсии: 16.732310566647637
Сижение дисперсии: 0.1859837017363919


In [25]:
from ambrosia.preprocessing import Cuped, MultiCuped, MLVarianceReducer

for i, df in enumerate(dfs):
    # our CUPAC
    transformer = CUPACTransformer(target_col='y')
    transformer.fit(df)
    transformed_data = transformer.transform(df)
    my_vr = transformer.variance_reduction

    # ambrosia MultiCuped
    cuped = MultiCuped(verbose=False)
    cuped.fit(
            df,
            'y',
            ["X1_lag", "X2_lag", "y0_lag_2"])
    df_cuped = cuped.transform(df)
    multicuped_vr = (1 - df_cuped['y_transformed'].var() / df['y'].var()) * 100

    # ambrosia MLVarianceReducer
    # ml = MLVarianceReducer()
    # ml.fit(df[['y', 'x1', 'x2', 'x3']])  # ML допускает больше фичей
    # df_ml = ml.transform(df[['y', 'x1', 'x2', 'x3']])
    # ml_vr = 1 - df_ml['y_cuped'].var() / df['y'].var()

    print(
        f"Dataset {i}: "
        f"My CUPAC VR={my_vr:.3f} | "
        f"MultiCuped VR={multicuped_vr:.3f} | "
        # f"MLVarianceReducer VR={ml_vr:.3f}"
    )


Dataset 0: My CUPAC VR=9.726 | MultiCuped VR=0.090 | 
Dataset 1: My CUPAC VR=0.231 | MultiCuped VR=0.110 | 
Dataset 2: My CUPAC VR=36.037 | MultiCuped VR=4.414 | 
Dataset 3: My CUPAC VR=16.509 | MultiCuped VR=0.998 | 
Dataset 4: My CUPAC VR=0.251 | MultiCuped VR=0.122 | 


In [26]:
cuped = MultiCuped()
cuped.fit(
        df,
        'y',
        ["X1_lag", "X2_lag", "y0_lag_1", "y0_lag_2"])
df_cuped = cuped.transform(df)
cuped_vr = (1 - df_cuped['y_transformed'].var() / df['y'].var()) * 100
print(cuped_vr)

ambrosia LOGGER: After transformation Multi СUPED for y, the variance is 69.2964 % of the original
ambrosia LOGGER: Variance transformation 11.5197 ===> 7.9828


30.70350703664263


In [27]:
transformer.lag_features   # список фич, которые шли в обучение

['X1_lag', 'X2_lag', 'y0_lag_2']

In [28]:
transformer.get_feature_mapping()  # словарь "старое имя" -> "новое"

{'X1_lag': 'X1', 'X2_lag': 'X2', 'y0_lag_2': 'y0_lag_1'}